# Capstone: Airbnb Price Listing Prediction
## Part 5 Production Model

_Authors: Evonne Tham_

___Example___

_In this notebook, I will calculate the r2 score and RMSE from the production model on the train set. I will then fit the same model on the test dataset to obtain the predicted sale prices, which will be used for the Kaggle submission. I will also state the insights gleaned from the model and subsequent business recommendations_

## Contents of this notebook
1. [Import Necessary Libraries and Load Data](#1.-Import-Necessary-Libraries-and-Load-Data)
2. [](#2.-)
3. [](#3.-)
4. [](#4.-) 


## 1. Import Necessary Libraries & Load Data </span>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# # modelling
# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# from sklearn.preprocessing import StandardScaler, MinMaxScaler 
# from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
# from sklearn.dummy import DummyRegressor

# from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
# from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score, accuracy_score
# import xgboost as xgb
# from xgboost import plot_importance


#Hide warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load in Data 
df = pd.read_csv('../datasets/final_df.csv')
print(f"Total Number of Listing: {df.shape[0]} | Total Number of Features: {df.shape[1]}")
df.head().T

--- 
## 2. Model Evaluation

The evaluation metrics used will be mean squared error (for loss) and r squared (for accuracy).